# DocTAPE Examples

DocTAPE (Documentation Testing and Automated Placement of Expressions) is a collection of utility functions (and wrappers for [Glue](https://myst-nb.readthedocs.io/en/latest/render/glue.html)) that are useful
for automating the process of building and testing documentation to ensure that documentation doesn't get stale.


In [ ]:
# Testing Cell
import inspect

from aviary.utils import doctape

imported_functions = {
    k
    for k, v in inspect.getmembers(doctape, inspect.isfunction)
    if v.__module__ == doctape.__name__
}
# We prepare all function names for using in the current page (before or later)
for func in imported_functions:
    doctape.glue_variable(func, md_code=True)
doctape.glue_variable(doctape.expected_error.__name__, md_code=True)

## Custom Classes

### {glue:md}`expected_error` 
Functions that raise an error provide the option to specify an error type to use instead of the default. This allows users to change the error type that is raised which can be useful in try/except blocks, especially when combined with the {glue:md}`expected_error` class.

In [ ]:
from aviary.utils.doctape import check_value, expected_error

try:
    check_value(int('1'), 2, error_type=expected_error)
except expected_error:
    print('we expected that to fail (1 is not equal to 2),')
print('but this will still run')

If we just used `ValueError` in the `except` branch, we might miss errors that we actually do want to catch.

In [ ]:
from aviary.utils.doctape import check_value, expected_error

try:
    check_value(int('1)'), 2)
except ValueError:
    print('1 is not equal to 2')
print("we mistyped '1', so we should have failed")

try:
    check_value(int('1)'), 2, error_type=expected_error)
except expected_error:
    print('1 is not equal to 2')
print("something unnexpected happened (we mistyped '1'), and we won't reach this")

## Testing Functions

The testing functions provide code that will raise errors when the documentation is built if the results don't match what is expected.
These can be used in places where it would be too difficult to glue portions of the documentation, or it is preferable to have a more uninterupted flow in the markdown cells.

However, it is important to note that it is possible to notice an error when the documentation builds and fix the code in the testing cell without updating the text in the markdown cell. For this reason, it is recommended to use a combination of testing and glueing functions in documentation.

### {glue:md}`check_value`
is a simple function for comparing two values.




In [ ]:
from aviary.models.missions.height_energy_default import phase_info
from aviary.utils.doctape import check_value

user_opts = phase_info['climb']['user_options']
check_value(user_opts['altitude_final'], (32000.0, 'ft'))
check_value(user_opts['mach_optimize'], False)

### {glue:md}`check_contains`
confirms that all the elements of one iterable are contained in the other

In [ ]:
import os

import aviary.api as av
from aviary.utils.doctape import check_contains

off_design_examples = av.get_path(os.path.join('examples'))
check_contains(
    ('run_off_design_example.py'),
    os.listdir(off_design_examples),
    error_string='{var} not in ' + str(off_design_examples),
    error_type=FileNotFoundError,
)
print('This file exists and does not raise any errors')
check_contains(
    ('made_up_file.py'),
    os.listdir(off_design_examples),
    error_string='{var} not in ' + str(off_design_examples),
    error_type=FileNotFoundError,
)
print('This file does not exist, so we will not reach this point')

Here we are checking that a certain file exists in a folder and specify a more useful error type than the default {glue:md}`default_error`

### {glue:md}`check_args`
gets the signature of a function and compares it to the arguments you are expecting.


In [ ]:
# Testing Cell
from aviary.utils.doctape import check_args, check_contains, glue_variable

default_error = RuntimeError
check_args(check_contains, {'error_type': default_error}, exact=False)
glue_variable('default_error', default_error.__name__)

exact_arg = 'exact'
check_args(check_args, exact_arg)
glue_variable(exact_arg, md_code=True)

Setting the {glue:md}`exact` argument to `False` means that we don't need to exactly match the signature of the function and instead just want to make sure that all of the arguments are valid and possibly that their default values are correct.

### {glue:md}`run_command_no_file_error`
executes a CLI command but won't fail if a FileNotFoundError is raised.

In [ ]:
# Testing Cell
from aviary.utils.doctape import run_command_no_file_error

command = """
    aviary run_mission --optimizer IPOPT --phase_info outputted_phase_info.py 
    validation_cases/benchmark_tests/test_aircraft/aircraft_for_bench_FwFm.csv
    --max_iter 0
"""
run_command_no_file_error(command)

This allows the command syntax and setup to be tested without requiring all of the files that command will use.

In [ ]:
# Testing Cell
import myst_nb

from aviary.utils.doctape import glue_variable

glue_variable(myst_nb.__name__)
glue_variable(myst_nb.glue.__name__, md_code=True)

## Glue Functions

The glue functions provide a wrapper for the {glue:md}`myst_nb` {glue:md}`glue` function that simplifies the interface.

After a variable has been glued in a Python cell, it can be accessed from a markdown cell with the \{glue:md\}\`variable name\` notation. Note that glue won't access the value of the glued variable until the documentation is built.

### {glue:md}`glue_variable`
allows users to specify a value that is {glue:}`something different than` what is displayed, but defaults to using the name of the variable if nothing is specified. This makes adapting old documentation easier, because users can just wrap {glue:}`the entire phrase they want to replace`.

Glued text can either be {glue:md}`plain text` or can be formatted as {glue:md}`inline code`. Note that the variables in this context do not have the same naming restrictions as Python variables, they can contain spaces or start with a number.


In [ ]:
# Testing Cell
from aviary.utils.doctape import glue_variable

glue_variable('plain text')
glue_variable('inline code', md_code=True)
glue_variable('something different than', 'not the same as')
glue_variable('the entire phrase they want to replace')

### {glue:md}`glue_keys` 
combines {glue:md}`get_all_keys` and {glue:md}`glue_variable` to glue all of the unique keys from a dict of dicts for later use.

In [ ]:
# Testing Cell
from aviary.utils.doctape import glue_keys

simplified_dict = {
    'phase1': {'altitude': {'val': 30, 'units': 'kft'}, 'mach': 0.4},
    'phase2': {'altitude': {'val': 10, 'units': 'km'}, 'mach': 0.5},
}
glue_keys(simplified_dict)

This allows us to ensure that {glue:md}`altitude` and {glue:md}`mach` do exist in the dictionary.

## Utility Functions

Utility functions are provided that the user may find useful for generating or testing their documentation.

### {glue:md}`gramatical_list`
is a simple function that forms a string that can be used in a sentence using a list of items.

In [ ]:
from aviary.utils.doctape import gramatical_list

single_element = gramatical_list([1])
two_elements = gramatical_list(['apples', 'bananas'])
three_elements_with_or = gramatical_list(['apples', 'bananas', 'strawberries'], 'or')

print(f'I would like to order {single_element} smoothie.')
print(f'Do you want {three_elements_with_or} in your smoothie?')
print(f'I only want {two_elements}.')

### {glue:md}`get_variable_name`
is a function that just returns the name of the variable passed to it as a string.

The contents of the variable can be of any type, as the variable isn't used in the function, but rather the [inspect](https://docs.python.org/3/library/inspect.html) functionality is used to retrieve the line of code itself.

{glue:md}`get_variable_name` can even accept multiple arguments, in which case a list of the names will be returned.

In [ ]:
from aviary.api import AviaryProblem
from aviary.utils.doctape import get_variable_name, glue_variable

glue_variable('function_name', get_variable_name(get_variable_name))
glue_variable(get_variable_name(print))

some_string = 'that contains important information'
simple_variable_name = get_variable_name(some_string)
phrase = simple_variable_name + ' is a variable ' + some_string
print(phrase)

complex_object_name = get_variable_name(AviaryProblem)
print(complex_object_name)

multiple = 2
arguments = str
print(get_variable_name(multiple, arguments))

{glue:md}`function_name` can be called directly in functions like {glue:md}`print` or {glue:md}`glue_variable` or the results can be saved.

### {glue:md}`get_previous_line`
returns the previous line of code as a string, which allows users to grab individual lines of code from Python cells to use as inline code in markdown cells.

In [ ]:
# Testing Cell
from aviary.api import Aircraft
from aviary.utils.doctape import get_previous_line, get_variable_name, glue_variable

glue_variable('value', Aircraft.Design.EMPTY_MASS, md_code=True)
glue_variable('var_value_code', get_previous_line(), md_code=True)
glue_variable(get_variable_name(Aircraft.Design.EMPTY_MASS), md_code=True)
glue_variable('var_name_code', get_previous_line(), md_code=True)

If you want to glue the name of a variable, instead of the value that variable holds, you can use the {glue:md}`get_variable_name` function to extract it.

For example:
Using {glue:md}`var_value_code` will result in {glue:md}`value`, whereas using {glue:md}`var_name_code` will result in {glue:md}`Aircraft.Design.EMPTY_MASS`

### {glue:md}`get_attribute_name`
allows users to get the name of object attributes in order to glue them into documentation. This works well for Enums or Class variables that have unique values.

In [ ]:
import aviary.api as av
from aviary.api import LegacyCode
from aviary.utils.doctape import get_attribute_name, glue_variable

some_custom_alias = av.LegacyCode

gasp_name = get_attribute_name(some_custom_alias, LegacyCode.GASP)
glue_variable(gasp_name)
brief_name = get_attribute_name(av.Verbosity, 1)
glue_variable(brief_name)
verbosity = get_attribute_name(av.Settings, av.Settings.VERBOSITY)
glue_variable(verbosity)
glue_variable(av.Settings.VERBOSITY)

###  {glue:md}`get_all_keys` and {glue:md}`get_value`
are intended to be used together for getting keys from nested dictionaries and then getting values back from those nested dictionaries, respectively. They were originally added for complex dictionaries, like the phase_info.

In [ ]:
from aviary.utils.doctape import get_all_keys, get_value

simplified_dict = {
    'phase1': {'altitude': {'val': 30, 'units': 'kft'}, 'mach': 0.4},
    'phase2': {'altitude': {'val': 10, 'units': 'km'}, 'mach': 0.5},
}
unique_keys_only = get_all_keys(simplified_dict)
all_keys = get_all_keys(simplified_dict, track_layers=True)
print(unique_keys_only)
print(all_keys)

p1_alt = get_value(simplified_dict, 'phase1.altitude.val')
print(p1_alt)

In [ ]:
# Testing Cell
from aviary.api import Aircraft, Mission
from aviary.utils.doctape import check_args, get_all_keys, get_previous_line, glue_variable

glue_variable(Aircraft.__name__)
glue_variable(Mission.__name__)

track_layers = 'track_layers'
check_args(get_all_keys, track_layers)
glue_variable(track_layers)

get_all_keys(Mission, track_layers='Mission')
track_layers_with_name = get_previous_line().split(', ')[1].split(')')[0]
glue_variable('track_layers_with_Mission', track_layers_with_name, display=False)

These can also be used to recursively get all of the attributes from a complex object, like the {glue:md}`Aircraft` or {glue:md}`Mission` hierarchies.


In [ ]:
from aviary.api import Mission
from aviary.utils.doctape import get_all_keys, get_value, glue_keys

k1 = get_all_keys(Mission)
print(k1[:5])  # Display the first 5 keys in Mission
k2 = get_all_keys(Mission, track_layers=True)
print(k2[:5])  # Display the first 5 keys in Mission
k3 = get_all_keys(Mission, track_layers='Mission')
print(k3[:5])  # Display the first 5 keys in Mission

glue_keys(Mission, False)

print(get_value(Mission, 'Constraints.GEARBOX_SHAFT_POWER_RESIDUAL'))

If {glue:md}`get_all_keys` is used on an object like {glue:md}`Mission` without specifying a value for {glue:md}`track_layers` will return all of the uniquely named attributes of the object (such as `GEARBOX_SHAFT_POWER_RESIDUAL`). Setting {glue:md}`track_layers` to `True` will get all of the attributes in dot notation, but will not include the name of the original object (`Constraints.GEARBOX_SHAFT_POWER_RESIDUAL`). If you want the full name of the attribute, including the name of the original object, you can use that name as the value of {glue:md}`track_layers` (using {glue:md}`track_layers_with_Mission` gives us access to {glue:md}`Mission.Constraints.GEARBOX_SHAFT_POWER_RESIDUAL`)

Using {glue:md}`glue_keys` handles this for us automatically by using the `__name__` attribute of the object passed to it as the value of {glue:md}`track_layers`.

As with the dict_of_dicts, we can recusively get the value of an attribute using the full path along with {glue:md}`get_value`.